# Python Testing Demo

## Why testing is valuable in Python

## Basic function testing

## Using fixtures

## HTTP Responses

## Why testing is valuable in Python

- Proof that you delivered what was expected from you
- Prevent unexpected behavior
- Document both the functionality as well as the behavior of the code
- Make sure that what worked still works after code changes


## Basic function testing

In [18]:
from IPython.display import Code

Code(filename='python_testing_demo/basic_functions.py', language='python')

def function_that_returns_text(text: str):
    if isinstance(text, str):
        return text
    else:
        raise ValueError(f"Input did not have a '{str}' type, but '{type(text)}'!")

In [19]:
from python_testing_demo.basic_functions import function_that_returns_text


print(function_that_returns_text("You should get this text back!"))

You should get this text back!


This code does exactly what it says it does, but you *proof* that it will by using tests to describe its behavior.

In [30]:
from IPython.display import Code

Code(filename='tests/basic_functions/test_basic_functions.py', language='python')

import pytest
from python_testing_demo.basic_functions import function_that_returns_text



def test_str():
    assert function_that_returns_text("You should see this text!") == "You should see this text!"


def test_int_as_str():
    assert function_that_returns_text("15") == "15"


@pytest.mark.issue
def test_int():
    assert function_that_returns_text(15) == "15"


@pytest.mark.fix
def test_int_should_fail():
    with pytest.raises(ValueError, match="Input did not have a"):
        assert function_that_returns_text(15) == "15"

In [4]:
!pytest tests/basic_functions/test_basic_functions.py -m "not fix" -v

============================= test session starts ==============================
platform linux -- Python 3.12.3, pytest-8.2.2, pluggy-1.5.0 -- /workspaces/python-testing-demo/.venv/bin/python
cachedir: .pytest_cache
rootdir: /workspaces/python-testing-demo
configfile: pytest.ini
plugins: anyio-4.4.0
collected 6 items / 3 deselected / 3 selected                                  

tests/basic_functions/test_basic_functions.py::test_str PASSED           [ 33%]
tests/basic_functions/test_basic_functions.py::test_int_as_str PASSED    [ 66%]
tests/basic_functions/test_basic_functions.py::test_int FAILED           [100%]

=================================== FAILURES ===================================
___________________________________ test_int ___________________________________

    @pytest.mark.issue
    def test_int():
>       assert function_that_returns_text(15) == "15"

tests/basic_functions/test_basic_functions.py:15: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

In [6]:
!pytest tests/basic_functions/test_basic_functions.py -m "fix" -v

============================= test session starts ==============================
platform linux -- Python 3.12.3, pytest-8.2.2, pluggy-1.5.0 -- /workspaces/python-testing-demo/.venv/bin/python
cachedir: .pytest_cache
rootdir: /workspaces/python-testing-demo
configfile: pytest.ini
plugins: anyio-4.4.0
collected 6 items / 3 deselected / 3 selected                                  

tests/basic_functions/test_basic_functions.py::test_int_should_fail PASSED [ 33%]
tests/basic_functions/test_basic_functions.py::test_function_that_returns_text[will_return_text] PASSED [ 66%]
tests/basic_functions/test_basic_functions.py::test_function_that_returns_text[will_fail_on_non-str_type] PASSED [100%]

======================= 3 passed, 3 deselected in 0.02s ========================


You might have noticed the usage of `@pytest.mark.issue` and `@pytest.mark.fix` as well as the flags `-m "not fix"` and `-m "fix"`: markers allow for easy filtering of tests to run/skip.

This is especially useful during various CI/CD stages, i.e.:

- Distinguish between slow and fast tests
- Select specific tests during different stages
- Filter tests that can only run in local/pipeline environments

> If you do not register custom markers in your `pytest.ini` file, you'll get a "unrecognized marker" warning.

Ref: https://docs.pytest.org/en/7.1.x/example/markers.html

In [35]:
!pytest --markers

@pytest.mark.issue: mark test as incorrectly failing.

@pytest.mark.fix: mark test as fixed test for issue.

@pytest.mark.anyio: mark the (coroutine function) test to be run asynchronously via anyio.

@pytest.mark.filterwarnings(warning): add a warning filter to the given test. see https://docs.pytest.org/en/stable/how-to/capture-warnings.html#pytest-mark-filterwarnings 

@pytest.mark.skip(reason=None): skip the given test function with an optional reason. Example: skip(reason="no way of currently testing this") skips the test.

@pytest.mark.skipif(condition, ..., *, reason=...): skip the given test function if any of the conditions evaluate to True. Example: skipif(sys.platform == 'win32') skips the test if we are on the win32 platform. See https://docs.pytest.org/en/stable/reference/reference.html#pytest-mark-skipif

@pytest.mark.xfail(condition, ..., *, reason=..., run=True, raises=None, strict=xfail_strict): mark the test function as an expected failure if any of the conditions eva

## Using fixtures

In [2]:
from IPython.display import Code

Code(filename='tests/fixtures/conftest.py', language='python')

import pytest

from python_testing_demo.fixtures import SomeClass

def pytest_configure():
    """
    Set up pytest global variables.
    """
    pytest.GLOBAL_TEST_VARIABLE = "test global variable"
    pytest.GLOBAL_TEST_VARIABLE_APPENDED = f"{pytest.GLOBAL_TEST_VARIABLE}_appended"



@pytest.fixture
def setup_and_teardown_class():
    """
    Sets up a reusable context for the actual tests and tears everything down afterwards.
    """

    # Anything that's set up here will be available during the lower-level test!
    print("Setting up the test context...")
    some_class = SomeClass()
    print(some_class.some_other_var)

    # Run the actual lower-level test
    yield some_class

    # Anything after the yield that was affected during the lower-level test is still in that same context!
    # Note that this can influence other tests if not teared down correctly
    print("Tearing down the test context...")
    print(some_class.some_other_var)



some_global_class = SomeClass()
@pytest.fixture
def incorrectly_setup_and_teardown_class():
    """
    Sets up a reusable context for the actual tests and tears everything down afterwards.
    """

    # Anything that's set up here will be available during the lower-level test!
    print("Setting up the test context...")
    some_class = some_global_class
    print(some_class.some_other_var)

    # Run the actual lower-level test
    yield some_class

    # Anything after the yield that was affected during the lower-level test is still in that same context!
    # Note that this can influence other tests if not teared down correctly
    print("Tearing down the test context...")
    print(some_class.some_other_var)

In [8]:
from IPython.display import Code

Code(filename='tests/fixtures/test_fixtures.py', language='python')

import pytest


def test_should_have_access_to_global_variable():
    assert pytest.GLOBAL_TEST_VARIABLE == "test global variable"
    assert "variable_appended" in pytest.GLOBAL_TEST_VARIABLE_APPENDED

In [15]:
!pytest tests/fixtures/test_fixtures.py -s -vvv

============================= test session starts ==============================
platform linux -- Python 3.12.3, pytest-8.2.2, pluggy-1.5.0 -- /workspaces/python-testing-demo/.venv/bin/python
cachedir: .pytest_cache
rootdir: /workspaces/python-testing-demo
configfile: pytest.ini
plugins: anyio-4.4.0
collected 4 items                                                              

tests/fixtures/test_fixtures.py::test_should_have_access_to_global_variable PASSED
tests/fixtures/test_fixtures.py::test_should_use_setup_and_teardown_and_overwrite_higher_level_var Setting up the test context...
bar
PASSEDTearing down the test context...
foo

tests/fixtures/test_fixtures.py::test_should_use_setup_and_teardown_and_overwrite_global_var Setting up the test context...
bar
PASSEDTearing down the test context...
foo

tests/fixtures/test_fixtures.py::test_should_get_overwritten_global_class Setting up the test context...
foo
PASSEDTearing down the test context...
foo


==============================

In [11]:
!jupyter nbconvert 'demo copy.ipynb' --to slides --post serve

[NbConvertApp] Converting notebook demo copy.ipynb to slides
[NbConvertApp] Writing 291181 bytes to demo copy.slides.html
[NbConvertApp] Redirecting reveal.js requests to https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.5.0
Serving your slides at http://127.0.0.1:8000/demo copy.slides.html
Use Control-C to stop this server
^C

Interrupted


### HTTP Responses

Sometimes you'll need to verify that certain HTTP requests contain specific data, i.e. to verify certain third-party behavior. You can catch and expose HTTP request data with the following code.

We use the out-of-the-box available pytest fixture `capsys` (https://docs.pytest.org/en/7.1.x/how-to/capture-stdout-stderr.html) together with our custom `debug_http` fixture.

In [1]:
from IPython.display import Code

Code(filename='tests/http/conftest.py', language='python')

import logging
import pytest


@pytest.fixture
def debug_http(scope="module", autouse=False):
    """
    Catches all out-going HTTP calls and prints them in the console.
    Use `autouse=True` to automatically use it in every test which imports this conftest.py file.
    """
    import http.client as http_client

    http_client.HTTPConnection.debuglevel = 1
    logging.basicConfig()
    logging.getLogger("debug_http").setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

In [12]:
from IPython.display import Code

Code(filename='tests/http/test_http.py', language='python')

import pytest
import requests


def test_http_logs_are_not_catched(
    capsys,
):
    requests.get("https://www.google.com/")

    out, err = capsys.readouterr()
    print(out)
    assert "reply: 'HTTP/1.1 200 OK\r\n'" not in out


def test_http_logs_are_catched(
    debug_http,
    capsys,
):
    requests.get("https://www.google.com/")

    out, err = capsys.readouterr()
    print(out)
    assert "reply: 'HTTP/1.1 200 OK\r\n'" in out

In [15]:
!pytest tests/http/test_http.py -s -vvv

============================= test session starts ==============================
platform linux -- Python 3.12.3, pytest-8.2.2, pluggy-1.5.0 -- /workspaces/python-testing-demo/.venv/bin/python
cachedir: .pytest_cache
rootdir: /workspaces/python-testing-demo
configfile: pyproject.toml
plugins: anyio-4.4.0
collected 2 items                                                              

tests/http/test_http.py::test_http_logs_are_not_catched 
PASSED
tests/http/test_http.py::test_http_logs_are_catched send: b'GET / HTTP/1.1\r\nHost: www.google.com\r\nUser-Agent: python-requests/2.32.3\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Date: Tue, 25 Jun 2024 13:43:11 GMT
header: Expires: -1
header: Cache-Control: private, max-age=0
header: Content-Type: text/html; charset=ISO-8859-1
header: Content-Security-Policy-Report-Only: object-src 'none';base-uri 'self';script-src 'nonce--Mjqk3TC2rmxHHHg91q7IA' 'strict-dynamic' 're